# Imports

## Import Libraries

In [1]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings

# Configure visualizations
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
%matplotlib inline

## Import Data

In [2]:
# loading data
df = pd.read_csv('../data/training_set_VU_DM.csv')
df['date_time'] = pd.to_datetime(df['date_time'])
df.head()
df_original = df.copy()

# Cleaning

In [3]:
duplicate_rows = df.duplicated().sum()
print(f"Number of duplicate rows: {duplicate_rows}")

Number of duplicate rows: 0


## Handling Missing Data

In [4]:
# Check for missing data in the entire dataframe
missing_data = df.isnull().sum()

# Show columns with missing data
missing_data = missing_data[missing_data > 0]
print(missing_data)

visitor_hist_starrating      4706481
visitor_hist_adr_usd         4705359
prop_review_score               7364
prop_location_score2         1090348
srch_query_affinity_score    4640941
orig_destination_distance    1607782
comp1_rate                   4838417
comp1_inv                    4828788
comp1_rate_percent_diff      4863908
comp2_rate                   2933675
comp2_inv                    2828078
comp2_rate_percent_diff      4402109
comp3_rate                   3424059
comp3_inv                    3307357
comp3_rate_percent_diff      4485550
comp4_rate                   4650969
comp4_inv                    4614684
comp4_rate_percent_diff      4827261
comp5_rate                   2735974
comp5_inv                    2598327
comp5_rate_percent_diff      4117248
comp6_rate                   4718190
comp6_inv                    4697371
comp6_rate_percent_diff      4862173
comp7_rate                   4642999
comp7_inv                    4601925
comp7_rate_percent_diff      4819832
c

In [5]:
# 1. Set all missing competitor fields to 0
comp_cols = [col for col in df.columns if col.startswith('comp')]
df[comp_cols] = df[comp_cols].fillna(0)

# 2. Impute 'orig_destination_distance' and 'prop_review_score' with their median
for col in ['orig_destination_distance', 'prop_review_score']:
    median_val = df[col].median()
    df[col] = df[col].fillna(median_val)

# 3. Impute 'srch_query_affinity_score' with the minimum value
df['srch_query_affinity_score'] = df['srch_query_affinity_score'].fillna(df['srch_query_affinity_score'].min())

# 4. Impute 'prop_location_score2' with the minimum per 'srch_destination_id'
df['prop_location_score2'] = df.groupby('srch_destination_id')['prop_location_score2'].transform(
    lambda x: x.fillna(x.min())
)
df['prop_location_score2'] = df['prop_location_score2'].fillna(df['prop_location_score2'].median())


# 5. Impute visitor historical features with mean
visitor_hist_cols = ['visitor_hist_starrating', 'visitor_hist_adr_usd']
for col in visitor_hist_cols:
    mean_val = df[col].mean()
    df[col] = df[col].fillna(mean_val)

# 6. Normalize 'price_usd' and 'prop_starrating' based on search and property groups

# Normalize 'price_usd' by 'srch_id'
df['price_usd_norm'] = df['price_usd'] / df.groupby('srch_id')['price_usd'].transform('mean')

# Normalize 'prop_starrating' by 'prop_id'
df['prop_starrating_norm'] = df['prop_starrating'] / df.groupby('prop_id')['prop_starrating'].transform('mean')
df['prop_starrating_norm'] = df['prop_starrating_norm'].fillna(0)

# 7. Drop the original 'price_usd' feature
df = df.drop(columns=['price_usd'])

# 8. Remove features not needed
# cols_to_drop = ['gross_bookings_usd', 'site_id', 'prop_id', 'srch_id']
cols_to_drop = ['gross_bookings_usd', 'position']
df = df.drop(columns=cols_to_drop)

In [6]:
missing_data = df.isnull().sum()
missing_data = missing_data[missing_data > 0]
print(missing_data)

Series([], dtype: int64)


In [7]:
df.to_csv('../data/clean_train.csv', index=False) 

## Handling class imbalance

In [ ]:
click_percentages = df['click_bool'].value_counts(normalize=True) * 100
print(click_percentages)

In [ ]:
# 1. Separate the two classes
clicked = df[df['click_bool'] == 1]
not_clicked = df[df['click_bool'] == 0]

# 2. Downsample the majority class (not clicked)
not_clicked_downsampled = not_clicked.sample(n=len(clicked), random_state=42)

# 3. Combine the two classes back together
df_balanced = pd.concat([clicked, not_clicked_downsampled])

# 4. Shuffle the resulting dataset
df_balanced = df_balanced.sample(frac=1, random_state=42).reset_index(drop=True)

In [ ]:
# Count original numbers
n_clicked = len(clicked)
n_not_clicked = len(not_clicked)

# Calculate how many will be dropped
rows_dropped = n_not_clicked - n_clicked

print(f"Rows with click_bool = 1: {n_clicked}")
print(f"Rows with click_bool = 0 before downsampling: {n_not_clicked}")
print(f"Rows to be dropped: {rows_dropped}")

In [ ]:
click_percentages = df_balanced['click_bool'].value_counts(normalize=True) * 100
print(click_percentages)

In [ ]:
click_counts = df_balanced['click_bool'].value_counts()
print(click_counts)